In [1]:
%load_ext autoreload 
%autoreload 2

from reranker import ReRanker
from opensearch_interface import OpenSearchClient
from typing import List, Dict
from rich import print
import os

from dotenv import load_dotenv
env_file = load_dotenv('./.env', override=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Bad pipe message: %s [b'\xd8\x19z\xb9\xb8k', b'\xda\x86\xe3\x85\xd8U6\xd1\xb9< \xe9\xb4h\xcf\xf2]\xe4L\x93\xc5', b'{\xd5\xa7\xed\xd9I\xde]=\xa6\x08\x8b\xb6\x83\xbc\x12\x82\xf6?s\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b', b'\x08\x05\x08\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xeb\xe7\xa1k=\xd0\x11k\xef\xbcr\x94\x14Q.\x83\x1b+\x86d\xc2\x04']
Bad pipe message: %s [b"BD\xca3\xc8A\x7f(\x90\xe7\xda\t*\x1d\x12WA\x02\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0

In [2]:
model_path = osclient = OpenSearchClient(model_name_or_path=os.environ['ST_MODEL_PATH'])

In [3]:
osclient.show_indexes()

health status index                              uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   kw-impact-theory                   2MjMun4bQYOoeUpv5UsJxg   3   1      33164            0     29.4mb         29.4mb
yellow open   semantic-impact-theory-196         SY2nXyvmQ9i5LAS4hmn82g   3   1      37007            0    694.6mb        694.6mb
yellow open   kw-impact-theory-196               vsuHausxRb6EjysQriOX5w   3   1      37007            0     30.5mb         30.5mb
yellow open   paul-graham3                       -74ZPvxoSMmtCPSzAI9o1A   1   1         18            0    768.2kb        768.2kb
yellow open   semantic-impact-theory-128         FJKOre3yT9aFxlF-_TvcTA   3   1      60380            0        1gb            1gb
yellow open   security-auditlog-2023.10.13       u6WOAjLiTIGdRVt8gJf_KA   1   1          3            0     46.4kb         46.4kb
green  open   .opendistro_security               _QeSqO4CQN2IU8VpE9hnPw   1   0         10

In [4]:
# osclient.indices.delete('climate-report')

In [5]:
def get_clean_results(search_hits: List[dict], keys: List[str]=['doc_id', 'title', 'content']) -> List[dict]:
    hits = [d['_source'] for d in search_hits]
    cleaned_hits = []
    for d in hits:
        hit = {k:v for k,v in d.items() if k in keys}
        cleaned_hits.append(hit)
    return cleaned_hits

### KW Search

In [6]:
index_name = "semantic-impact-theory-196"

In [7]:
query = "How to become a millionaire?"

In [8]:
res = osclient.keyword_search(query, index_name, 5)
results = get_clean_results(res)
print(results)

[
    {
        'title': 'How Transparency Wins the Day In Finance and In Life | Tim Sykes on Impact Theory',
        'doc_id': 'Zetsm0aTdME_46',
        'content': "I find that when I just do original work and myself, people gravitate towards me. If someone 
really is going to be a good student, they're going to need to be so dedicated. If someone is like, oh, he swears 
or he has a grammar problem. There's a million criticisms that I've seen. I'm just like, these are such meaningless
criticisms. In order to be successful, I think this is the biggest thing. You have to be so dedicated to your craft
and so passionate about it and so single-minded focused on that task. Most people just don't take it that 
seriously. They're like, yeah, I want to be a millionaire. Yeah. I meet people and they're like, oh, you're that 
millionaire. A smile comes on their face like, oh, how do I become a millionaire? They don't really believe it."
    },
    {
        'title': 'Silicon Valley Bank MELTDOWN Explained | How to PREPARE for the RECESSION | Jaspreet Singh',
        'doc_id': 'dVO-7rchstw_108',
        'content': "Now, if I could give people that mindset, an overwhelming number of them are still going to 
fail. But you create an environment like the U.S. where, man, really amazing people from all over the world want to
come here so that they're not strucken down by other people who don't want to see a tall poppy. They don't want to 
see somebody stand up above other people. They're cheering for it. They want to do it themselves, and it's become a
meme. This really pisses me off. It's become a meme, the whole idea of people voting against their own interest 
because they're just a temporarily embarrassed millionaire. That was my entire youth until I was in my late 30s. I 
acted like the temporarily embarrassed millionaire, and it led me to becoming an actual millionaire."
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_61',
        'content': "You don't understand how deeply rooted your wire Circuitry is you got to rewire it and it takes
reps instead of reps and sets It takes hours and days or days and years and so I think that if you're young 
watching this Don't try to become a millionaire too young Because what you should do when you're young is work on 
the circuitry part And then the money will come Alan nation told me that he said I don't try to become a 
millionaire in your 20s And I didn't listen to him and I wish I had and luckily I kind of listened to him but I 
should have listened more and so if you could take those formative years and just Go on the adventure of life 
That's why I went to India and a leper colony and went and live in the Amish because I was like Amish all the good 
Stuff you have reprogram my brain."
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_20',
        'content': "So you get to walk 24 Thousand miles and you'll come up on the backside and you will get your 
goal That's most entrepreneurs the average person takes 20 years to become a millionaire 90% of businesses fail 
within the first five years 80 to 90 depending on what statistic Most people I did the math once the average 
American has $60,000 saved by the time. They're about 60 years old So my answer I did the math you can do this with
a simple financial calculator Everybody in America your parents everybody, you know will be a millionaire If they 
live to 160 at 160 years old You take 60 grand at age 60 and you give it a decent return on investment 8% 10% 
You'll be a millionaire at 160, but the problem is the great philosopher I think was Aristotle Socrates said the 
problem is Art is long But life is short the art of living and getting to your objective is Long,"
    },
    {
        'title': 'Young Entrepreneur on How He Built A Multimillion-Dollar Business | Steven Bartlett on Impact 
Theory',
        

### Semantic Search

In [9]:
response = osclient.vector_search(query, index_name, size=5)
clean_response = get_clean_results(response)
print(clean_response)

[
    {
        'title': 'What Everyone Who is Too Self-Critical Needs to Know | Marisa Peer on Impact Theory',
        'doc_id': 'wT9X6dD8j0k_27',
        'content': "You can't tell yourself you're going to be an astronaut. And when people say to me, I want to 
be a millionaire, I'm like, what are you going to do to be a millionaire? You can't go, I'm going to sit at home. 
And that's the problem with the secret, attract millions to you. What are you going to do for these millions? I 
want to be, have a six-pack. You do need to go to the gym. I want to have great health. Well, stop eating pizza and
eat better food. But once you get the mind to want it, it's actually easy. But when people say to me, I want to be 
a millionaire, then we tell them, well, any goal of significance requires you to learn something new. But then they
have a fear of learning. And of course, you're right."
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_20',
        'content': "So you get to walk 24 Thousand miles and you'll come up on the backside and you will get your 
goal That's most entrepreneurs the average person takes 20 years to become a millionaire 90% of businesses fail 
within the first five years 80 to 90 depending on what statistic Most people I did the math once the average 
American has $60,000 saved by the time. They're about 60 years old So my answer I did the math you can do this with
a simple financial calculator Everybody in America your parents everybody, you know will be a millionaire If they 
live to 160 at 160 years old You take 60 grand at age 60 and you give it a decent return on investment 8% 10% 
You'll be a millionaire at 160, but the problem is the great philosopher I think was Aristotle Socrates said the 
problem is Art is long But life is short the art of living and getting to your objective is Long,"
    },
    {
        'title': 'The 5 Things BROKE People Do That The Rich DON’T DO | Tom Bilyeu',
        'doc_id': 'OoGghm0_Q8I_13',
        'content': "Keep walking towards your goal. Well, the truth is if you go south, when you should go north, 
you could have gone one mile, but the earth is about 24,000 miles in circumference. So you get to walk 24,000 miles
and you'll come up on the backside and you will get your goal. That's most entrepreneurs. The average person takes 
20 years to become a millionaire. 90% of businesses fail within the first five years, 80 to 90 depending on what 
statistic. Most people, I did the math once, the average American has $60,000 saved by the time they're about 60 
years old. So my answer, I did the math. You can do this with a simple financial calculator. Everybody in America, 
your parents, everybody you know will be a millionaire if they live to 160."
    },
    {
        'title': 'How Transparency Wins the Day In Finance and In Life | Tim Sykes on Impact Theory',
        'doc_id': 'Zetsm0aTdME_46',
        'content': "I find that when I just do original work and myself, people gravitate towards me. If someone 
really is going to be a good student, they're going to need to be so dedicated. If someone is like, oh, he swears 
or he has a grammar problem. There's a million criticisms that I've seen. I'm just like, these are such meaningless
criticisms. In order to be successful, I think this is the biggest thing. You have to be so dedicated to your craft
and so passionate about it and so single-minded focused on that task. Most people just don't take it that 
seriously. They're like, yeah, I want to be a millionaire. Yeah. I meet people and they're like, oh, you're that 
millionaire. A smile comes on their face like, oh, how do I become a millionaire? They don't really believe it."
    },
    {
        'title': 'Silicon Valley Bank MELTDOWN Explained | How to PREPARE for the RECESSION | Jaspreet Singh',
        'doc_id': 'dVO-7rchstw_109',
        'content': "I acted like the temporarily embarrassed millionaire, and it

### Hybrid Search

In [10]:
hybrid = osclient.hybrid_search(query, index_name, index_name)
clean_hybrid = get_clean_results(hybrid)
print(clean_hybrid[:5])

[
    {
        'title': 'How Transparency Wins the Day In Finance and In Life | Tim Sykes on Impact Theory',
        'doc_id': 'Zetsm0aTdME_46',
        'content': "I find that when I just do original work and myself, people gravitate towards me. If someone 
really is going to be a good student, they're going to need to be so dedicated. If someone is like, oh, he swears 
or he has a grammar problem. There's a million criticisms that I've seen. I'm just like, these are such meaningless
criticisms. In order to be successful, I think this is the biggest thing. You have to be so dedicated to your craft
and so passionate about it and so single-minded focused on that task. Most people just don't take it that 
seriously. They're like, yeah, I want to be a millionaire. Yeah. I meet people and they're like, oh, you're that 
millionaire. A smile comes on their face like, oh, how do I become a millionaire? They don't really believe it."
    },
    {
        'title': 'What Everyone Who is Too Self-Critical Needs to Know | Marisa Peer on Impact Theory',
        'doc_id': 'wT9X6dD8j0k_27',
        'content': "You can't tell yourself you're going to be an astronaut. And when people say to me, I want to 
be a millionaire, I'm like, what are you going to do to be a millionaire? You can't go, I'm going to sit at home. 
And that's the problem with the secret, attract millions to you. What are you going to do for these millions? I 
want to be, have a six-pack. You do need to go to the gym. I want to have great health. Well, stop eating pizza and
eat better food. But once you get the mind to want it, it's actually easy. But when people say to me, I want to be 
a millionaire, then we tell them, well, any goal of significance requires you to learn something new. But then they
have a fear of learning. And of course, you're right."
    },
    {
        'title': 'Silicon Valley Bank MELTDOWN Explained | How to PREPARE for the RECESSION | Jaspreet Singh',
        'doc_id': 'dVO-7rchstw_108',
        'content': "Now, if I could give people that mindset, an overwhelming number of them are still going to 
fail. But you create an environment like the U.S. where, man, really amazing people from all over the world want to
come here so that they're not strucken down by other people who don't want to see a tall poppy. They don't want to 
see somebody stand up above other people. They're cheering for it. They want to do it themselves, and it's become a
meme. This really pisses me off. It's become a meme, the whole idea of people voting against their own interest 
because they're just a temporarily embarrassed millionaire. That was my entire youth until I was in my late 30s. I 
acted like the temporarily embarrassed millionaire, and it led me to becoming an actual millionaire."
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_20',
        'content': "So you get to walk 24 Thousand miles and you'll come up on the backside and you will get your 
goal That's most entrepreneurs the average person takes 20 years to become a millionaire 90% of businesses fail 
within the first five years 80 to 90 depending on what statistic Most people I did the math once the average 
American has $60,000 saved by the time. They're about 60 years old So my answer I did the math you can do this with
a simple financial calculator Everybody in America your parents everybody, you know will be a millionaire If they 
live to 160 at 160 years old You take 60 grand at age 60 and you give it a decent return on investment 8% 10% 
You'll be a millionaire at 160, but the problem is the great philosopher I think was Aristotle Socrates said the 
problem is Art is long But life is short the art of living and getting to your objective is Long,"
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_61',
        'content': "You don't understand how deeply rooted your wi

### Reranking

In [11]:
reranker = ReRanker()

In [12]:
ranked_res = reranker.rerank(hybrid, query, top_k=5)

In [13]:
print(get_clean_results(ranked_res))

[
    {
        'title': 'The 5 Things BROKE People Do That The Rich DON’T DO | Tom Bilyeu',
        'doc_id': 'OoGghm0_Q8I_13',
        'content': "Keep walking towards your goal. Well, the truth is if you go south, when you should go north, 
you could have gone one mile, but the earth is about 24,000 miles in circumference. So you get to walk 24,000 miles
and you'll come up on the backside and you will get your goal. That's most entrepreneurs. The average person takes 
20 years to become a millionaire. 90% of businesses fail within the first five years, 80 to 90 depending on what 
statistic. Most people, I did the math once, the average American has $60,000 saved by the time they're about 60 
years old. So my answer, I did the math. You can do this with a simple financial calculator. Everybody in America, 
your parents, everybody you know will be a millionaire if they live to 160."
    },
    {
        'title': "Tai Lopez on Why Grinding Isn't Enough | Impact Theory",
        'doc_id': 'jX5eajzLJMU_20',
        'content': "So you get to walk 24 Thousand miles and you'll come up on the backside and you will get your 
goal That's most entrepreneurs the average person takes 20 years to become a millionaire 90% of businesses fail 
within the first five years 80 to 90 depending on what statistic Most people I did the math once the average 
American has $60,000 saved by the time. They're about 60 years old So my answer I did the math you can do this with
a simple financial calculator Everybody in America your parents everybody, you know will be a millionaire If they 
live to 160 at 160 years old You take 60 grand at age 60 and you give it a decent return on investment 8% 10% 
You'll be a millionaire at 160, but the problem is the great philosopher I think was Aristotle Socrates said the 
problem is Art is long But life is short the art of living and getting to your objective is Long,"
    },
    {
        'title': 'What Everyone Who is Too Self-Critical Needs to Know | Marisa Peer on Impact Theory',
        'doc_id': 'wT9X6dD8j0k_27',
        'content': "You can't tell yourself you're going to be an astronaut. And when people say to me, I want to 
be a millionaire, I'm like, what are you going to do to be a millionaire? You can't go, I'm going to sit at home. 
And that's the problem with the secret, attract millions to you. What are you going to do for these millions? I 
want to be, have a six-pack. You do need to go to the gym. I want to have great health. Well, stop eating pizza and
eat better food. But once you get the mind to want it, it's actually easy. But when people say to me, I want to be 
a millionaire, then we tell them, well, any goal of significance requires you to learn something new. But then they
have a fear of learning. And of course, you're right."
    },
    {
        'title': 'Silicon Valley Bank MELTDOWN Explained | How to PREPARE for the RECESSION | Jaspreet Singh',
        'doc_id': 'dVO-7rchstw_109',
        'content': "I acted like the temporarily embarrassed millionaire, and it led me to becoming an actual 
millionaire. If you see yourself as like, this is all you're ever going to be, and you're stuck, then you won't do 
the things you need to do to at least have a shot. A lot of people are still going to fail. I get that. But man, if
you think, oh, it's what I call the only belief that matters. If you believe, if I put time and energy into getting
better at this thing, the world will still punish me. Rich white people are going to take it away. I will never get
anywhere that I want to go. You won't do the things."
    },
    {
        'title': 'How Transparency Wins the Day In Finance and In Life | Tim Sykes on Impact Theory',
        'doc_id': 'Zetsm0aTdME_50',
        'content': "This wasn't available, but now we have so many little mini pieces of content and any one of 
them can touch any one person or any one of them can touch a hundred people or a thousand people and make them 
think a little differ